# 1. Preparation

In [1]:
import setproctitle

setproctitle.setproctitle("python3")
import os
import time
import sys
import chromadb
import bm25s
import Stemmer
import numpy as np

sys.path.append("../..")

from tqdm import tqdm
from transformers import set_seed
from sentence_transformers import SentenceTransformer
from chromadb.api.models.Collection import Collection
from benchmark_generator.context.utils.jsonl import read_jsonl


os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
set_seed(42, deterministic=True)

/raid/p-storage/luthfibalaka_d2b65c57/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
stemmer = Stemmer.Stemmer("english")
reranker = SentenceTransformer("../../models/stella", local_files_only=True)

In [3]:
dataset = "fetaqa"
if dataset == "chicago":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/chicago_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/chicago.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_chicago_10K_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_chicago_10K"
elif dataset == "public":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/public_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/public.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_public_bi_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_public_bi"
elif dataset == "chembl":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/chembl_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/chembl.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_chembl_10K_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_chembl_10K"
elif dataset == "adventure":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/adventure_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/adventure.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_adventure_works_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_adventure_works"
elif dataset == "fetaqa":
    narrations = read_jsonl("../../pneuma_summarizer/summaries/narrations/fetaqa_narrations.jsonl")
    rows = read_jsonl("../../pneuma_summarizer/summaries/rows/fetaqa.jsonl")
    content_benchmark = read_jsonl("../../data_src/benchmarks/content/pneuma_fetaqa_questions_annotated.jsonl")
    path = "../../data_src/tables/pneuma_fetaqa"

# 2. Indexing

In [4]:
def indexing_keyword(
    stemmer,
    narration_contents: list[dict[str, str]],
    contexts: list[dict[str, str]] = None,
):
    corpus_json = []
    tables = sorted({content["table"] for content in narration_contents})
    for table in tables:
        cols_descriptions = [
            content["summary"]
            for content in narration_contents
            if content["table"] == table
        ]
        for content_idx, content in enumerate(cols_descriptions):
            corpus_json.append(
                {
                    "text": content,
                    "metadata": {"table": f"{table}_SEP_contents_{content_idx}"},
                }
            )

        if contexts is not None:
            filtered_contexts = [
                context["context"] for context in contexts if context["table"] == table
            ]
            for context_idx, context in enumerate(filtered_contexts):
                corpus_json.append(
                    {
                        "text": context,
                        "metadata": {"table": f"{table}_SEP_{context_idx}"},
                    }
                )

    corpus_text = [doc["text"] for doc in corpus_json]
    corpus_tokens = bm25s.tokenize(
        corpus_text, stopwords="en", stemmer=stemmer, show_progress=False
    )

    retriever = bm25s.BM25(corpus=corpus_json)
    retriever.index(corpus_tokens, show_progress=False)
    return retriever

In [5]:
print(f"Processing {dataset} dataset")
start = time.time()
client = chromadb.PersistentClient(
    f"../indices/index-{dataset}-content-rows-narrations"
)
collection = client.get_collection("benchmark")
retriever = indexing_keyword(stemmer, rows + narrations)
end = time.time()
print(f"Indexing time: {end-start} seconds")

Processing fetaqa dataset


# 3. Benchmarking

In [6]:
def process_nodes_bm25(items):
    # Normalize relevance scores and return the nodes in dict format
    results, scores = items
    scores: list[float] = scores[0]
    max_score = max(scores)
    min_score = min(scores)

    processed_nodes: dict[str, tuple[float, str]] = {}
    for i, node in enumerate(results[0]):
        if min_score == max_score:
            score = 1
        else:
            score = (scores[i] - min_score) / (max_score - min_score)
        processed_nodes[node["metadata"]["table"]] = (score, node["text"])
    return processed_nodes

In [7]:
def process_nodes_vec(items):
    # Normalize relevance scores and return the nodes in dict format
    scores: list[float] = [1 - dist for dist in items["distances"][0]]
    max_score = max(scores)
    min_score = min(scores)

    processed_nodes: dict[str, tuple[float, str]] = {}

    for idx in range(len(items["ids"][0])):
        if min_score == max_score:
            score = 1
        else:
            score = (scores[idx] - min_score) / (max_score - min_score)
        processed_nodes[items["ids"][0][idx]] = (score, items["documents"][0][idx])
    return processed_nodes

In [8]:
def hybrid_retriever(
    bm25_res,
    vec_res,
    k: int,
    question: str,
    use_reranker=False,
):
    processed_nodes_bm25 = process_nodes_bm25(bm25_res)
    processed_nodes_vec = process_nodes_vec(vec_res)

    node_ids = set(list(processed_nodes_bm25.keys()) + list(processed_nodes_vec.keys()))
    all_nodes: list[tuple[str, float, str]] = []
    for node_id in sorted(node_ids):
        bm25_score_doc = processed_nodes_bm25.get(node_id, (0.0, None))
        vec_score_doc = processed_nodes_vec.get(node_id, (0.0, None))

        combined_score = 0.5 * bm25_score_doc[0] + 0.5 * vec_score_doc[0]
        if bm25_score_doc[1] is None:
            doc = vec_score_doc[1]
        else:
            doc = bm25_score_doc[1]

        all_nodes.append((node_id, combined_score, doc))

    sorted_nodes = sorted(all_nodes, key=lambda node: (-node[1], node[0]))[:k]
    if use_reranker:
        sorted_nodes = rerank(sorted_nodes, question)
    return sorted_nodes

In [9]:
from scipy.spatial.distance import cosine
def rerank(nodes: list[tuple[str, float, str]], question: str):
    # Each node is of the form (name, score, doc)
    names = []
    docs = []
    for node in nodes:
        names.append(node[0])
        docs.append(node[2])

    docs_embeddings = reranker.encode(
        docs,
        batch_size=100,
        device="cuda",
    )
    question_embedding = reranker.encode(question, device="cuda")
    similarities = [
        1 - cosine(question_embedding, docs_embedding) for docs_embedding in docs_embeddings
    ]

    reranked_nodes = sorted(zip(names, similarities, docs), key=lambda x: (-x[1], x[0]))
    return reranked_nodes

In [10]:
def get_question_key(benchmark_type: str, use_rephrased_questions: bool):
    if benchmark_type == "content":
        if not use_rephrased_questions:
            question_key = "question_from_sql_1"
        else:
            question_key = "question"
    else:
        if not use_rephrased_questions:
            question_key = "question_bx1"
        else:
            question_key = "question_bx2"
    return question_key

In [11]:
def evaluate_benchmark(
    benchmark: list[dict[str, str]],
    benchmark_type: str,
    k: int,
    collection: Collection,
    retriever,
    stemmer,
    n=3,
    use_reranker=False,
    use_rephrased_questions=False,
):
    start = time.time()
    hitrate_sum = 0
    wrong_questions = []

    if use_reranker:
        increased_k = k * n
    else:
        increased_k = k * n

    question_key = get_question_key(benchmark_type, use_rephrased_questions)

    questions = []
    for data in benchmark:
        questions.append(data[question_key])
    embed_questions = np.loadtxt(
        f"../embeddings/embed-{dataset}-questions-{benchmark_type}-{use_rephrased_questions}.txt"
    )
    embed_questions = [embed.tolist() for embed in embed_questions]

    for idx, datum in enumerate(tqdm(benchmark)):
        answer_tables = datum["answer_tables"]
        question_embedding = embed_questions[idx]

        query_tokens = bm25s.tokenize(
            questions[idx], stemmer=stemmer, show_progress=False
        )
        results, scores = retriever.retrieve(
            query_tokens, k=increased_k, show_progress=False
        )
        bm25_res = (results, scores)

        vec_res = collection.query(
            query_embeddings=[question_embedding], n_results=increased_k
        )

        all_nodes = hybrid_retriever(
            bm25_res, vec_res, increased_k, questions[idx], use_reranker
        )
        before = hitrate_sum
        for table, _, _ in all_nodes[:k]:
            table = table.split("_SEP_")[0]
            if table in answer_tables:
                hitrate_sum += 1
                break
        if before == hitrate_sum:
            wrong_questions.append(idx)
        # Checkpoint
        if idx % 20 == 0:
            print(f"Current Hit Rate Sum at index {idx}: {hitrate_sum}")
            print(
                f"Current wrongly answered questions at index {idx}: {wrong_questions}"
            )

    end = time.time()
    print(f"Hit Rate: {round(hitrate_sum/len(benchmark) * 100, 2)}")
    print(f"Benchmarking Time: {end - start} seconds")
    print(f"Wrongly answered questions: {wrong_questions}")

In [12]:
ks = [1]
ns = [5]
use_reranker = True

In [ ]:
for k in ks:
    for n in ns:
        print(f"BC1 with k={k} n={n}")
        evaluate_benchmark(
            content_benchmark,
            "content",
            k,
            collection,
            retriever,
            stemmer,
            n=n,
            use_reranker=use_reranker,
        )
        print("=" * 50)

In [ ]:
for k in ks:
    for n in ns:
        print(f"BC2 with k={k} n={n}")
        evaluate_benchmark(
            content_benchmark,
            "content",
            k,
            collection,
            retriever,
            stemmer,
            n=n,
            use_rephrased_questions=True,
            use_reranker=use_reranker,
        )
    print("=" * 50)